You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the current landscape of artificial intelligence
- Importance of staying informed about the latest trends and key players in the industry

Key Points:
1. Latest Trends in Artificial Intelligence:
- Breakthroughs in natural language processing and understanding
- Advancements in computer vision and image recognition
- Rise of AI-powered decision-making systems

2. Key Players in the Artific

I now can give a great answer

Final Answer:

# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction:**

Artificial Intelligence (AI) has seamlessly integrated into our daily routines, from the convenience of smart assistants on our smartphones to the tailored recommendations on streaming services. It is imperative to stay abreast of the latest trends and prominent figures in the industry to grasp the profound impact of AI on both society and business. This blog post delves into the current landscape of AI, spotlighting the prevailing trends, key players, and notable news that are shaping the future of artificial intelligence.

**Latest Trends in Artificial Intelligence:**

One of the most noteworthy trends in AI revolves around the breakthroughs in natural language processing and comprehension. This advancement has paved the way for the creation of chatbots and virtual assistants capable of engaging with users in a remarkably human-like manner. Furthermore,

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction:**

Artificial Intelligence (AI) has seamlessly integrated into our daily routines, from the convenience of smart assistants on our smartphones to the tailored recommendations on streaming services. It is imperative to stay abreast of the latest trends and prominent figures in the industry to grasp the profound impact of AI on both society and business. This blog post delves into the current landscape of AI, spotlighting the prevailing trends, key players, and notable news that are shaping the future of artificial intelligence.

**Latest Trends in Artificial Intelligence:**

One of the most noteworthy trends in AI revolves around the breakthroughs in natural language processing and comprehension. This advancement has paved the way for the creation of chatbots and virtual assistants capable of engaging with users in a remarkably human-like manner. Furthermore, strides in computer vision and image recognition have empowered AI systems to analyze and interpret visual data with unparalleled precision. Another significant trend is the emergence of AI-powered decision-making systems that are streamlining processes and enhancing efficiency across various industries.

**Key Players in the Artificial Intelligence Industry:**

Google's DeepMind stands at the forefront of AI research, making substantial contributions to the field. Their pioneering work in reinforcement learning and neural networks has pushed the boundaries of AI capabilities. Similarly, IBM Watson is a pivotal player in the industry, particularly in sectors like healthcare and finance. Their AI-driven solutions are revolutionizing the operational landscape of these industries. Tesla's utilization of AI in autonomous driving technology is also commendable, as they continuously push the envelope of what is achievable with AI in the automotive sector.

**Noteworthy News in Artificial Intelligence:**

Recent developments in AI ethics and regulations have ignited crucial dialogues on the responsible application of AI technology. The impact of AI on job automation and workforce displacement remains a prominent concern, with discussions surrounding the future of work in an AI-centric world. Collaboration between tech behemoths and startups in AI innovation is propelling the industry forward, ushering in a wave of fresh and stimulating advancements on the horizon.

**Conclusion:**

With the rapid progression of AI, keeping abreast of the latest trends and key players in the industry is indispensable. By staying informed on the developments within artificial intelligence, we can gain a deeper understanding of its implications on society and business alike. Stay tuned to our blog for regular updates on AI trends and news, and feel free to share your insights and queries on artificial intelligence in the comments section. Let's embark on this journey of exploring the future of artificial intelligence together.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "How to holistically evaluate LLMs accuracy and reliability in patient facing apps"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on How to holistically evaluate LLMs accuracy and reliability in patient facing apps.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document

Title: How to holistically evaluate LLMs accuracy and reliability in patient facing apps

Introduction:
- Brief overview of the importance of accurately evaluating LLMs in patient facing apps
- Explanation of the significance of reliability and accuracy in healthcare technology

Key Points:
1. Latest Trends:
- Incorporation of machine learning models in healthcare technology
- Emphasis on the need for accurate and reliable LLMs i

I now can give a great answer

Final Answer:
# How to Holistically Evaluate LLMs Accuracy and Reliability in Patient-Facing Apps

In the rapidly evolving landscape of healthcare technology, the integration of machine learning models (LLMs) in patient-facing apps has become increasingly prevalent. The accurate and reliable performance of these LLMs is crucial in ensuring optimal patient care and outcomes. As the demand for healthcare technology continues to grow, it is imperative to have robust mechanisms in place to evaluate the accuracy and reliability of these models effectively.

The incorporation of machine learning models in healthcare technology represents a significant advancement in the field. The emphasis on the need for accurate and reliable LLMs in patient-facing apps is driven by the desire to enhance the quality of patient care. With continuous advancements in technology for evaluating LLMs accuracy, there is a growing focus on developing more sophisticated tools and metho

In [14]:
Markdown(result)

# How to Holistically Evaluate LLMs Accuracy and Reliability in Patient-Facing Apps

In the rapidly evolving landscape of healthcare technology, the integration of machine learning models (LLMs) in patient-facing apps has become increasingly prevalent. The accurate and reliable performance of these LLMs is crucial in ensuring optimal patient care and outcomes. As the demand for healthcare technology continues to grow, it is imperative to have robust mechanisms in place to evaluate the accuracy and reliability of these models effectively.

The incorporation of machine learning models in healthcare technology represents a significant advancement in the field. The emphasis on the need for accurate and reliable LLMs in patient-facing apps is driven by the desire to enhance the quality of patient care. With continuous advancements in technology for evaluating LLMs accuracy, there is a growing focus on developing more sophisticated tools and methodologies to assess their performance.

Leading healthcare technology companies are investing heavily in the development of LLMs to improve patient outcomes. Experts in the field of machine learning and healthcare technology are at the forefront of innovation, driving progress in evaluating the accuracy and reliability of these models. Regulatory bodies play a crucial role in ensuring that patient-facing apps meet standards for accuracy and reliability, safeguarding the interests of both healthcare professionals and patients.

Recent case studies have shed light on the significant impact of inaccurate LLMs on patient outcomes, highlighting the importance of rigorous evaluation processes. Updates on regulations and guidelines for evaluating LLMs accuracy provide valuable insights for developers and designers of patient-facing apps. Success stories of patient-facing apps with reliable LLMs serve as inspiration for the industry, showcasing the potential benefits of implementing accurate and reliable models.

In conclusion, the evaluation of LLMs accuracy and reliability in patient-facing apps is a critical aspect of healthcare technology that requires careful consideration. By staying informed about the latest trends and guidelines, healthcare professionals, developers, patients, and policymakers can work together to ensure the effective implementation of accurate LLMs. With a collaborative effort and a commitment to quality, the healthcare industry can harness the power of machine learning to improve patient care and outcomes.

Don't forget to check out the resources provided by organizations such as the American Medical Association, Healthcare Information and Management Systems Society, and the National Institute of Standards and Technology for further insights into evaluating LLMs accuracy. Stay engaged in the conversation and continue to explore new possibilities in healthcare technology.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).